## Domácí úkol - Batoh

Za domácí úkol budete mít vyřešit pomocí evolučního algoritmu problém batohu. Ten spočívá v tom, že máme batoh kapacity K a N předmětů, každý s cenou c<sub>i</sub> a objemem v<sub>i</sub> a chceme vybrat takové věci, abychom maximalizovali zisk a zároveň abychom nepřekročili kapacitu batohu. 

Vstupní data máte ve složce *domaci_ukol_data*. Obsahuje čtyři soubory s daty a dva s výsledky. Na první řádce souboru s daty je vždy počet předmětů a kapacita batohu oddělené mezerou, každý další následující řádek obsahuje cenu a objem předmětu taktéž oddělené mezerou. První dva soubory slouží pro snažší odladění evolučního algoritmu a obsahují i k sobě extra soubory s optimálním řešením. Na dalších dvou máte za úkol algoritmus pustit a výsledky na nich naměřené mi poslat. 

Napište tedy nějaký svůj evoluční algoritmus, který bude řešit problém batohu a pusťte ho na vstupních datech. Svůj kód, popis evolučního algoritmu (zvolené evoluční operátory, kódování jedince, atd.) a rozbor výsledků, včetně nejlepšího dosaženého skóre i s jejich odůvodněním mi pošlete emailem do stanoveného deadline.  Pro sepsání popisu vašeho evolučního algoritmu, parametrů evoluce, zvolené reprezentace jedince a rozboru výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/04_spojita_reprezentace/DU1_evolucni_algoritmy.pdf).

##### Importy

In [2]:
import array
import random
import numpy as np
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import my_algorithms

Inicializácia random itemov a veľkosť batohu

In [3]:
# random.seed(64)
random.seed()
items_count = 20

weight_bounds = (10,200)
price_bounds = (10,200)

weights = random.sample(range(weight_bounds[0],weight_bounds[1]), items_count)
prices = random.sample(range(price_bounds[0],price_bounds[1]), items_count)

max_weight = 1000

Načítanie batohu zo súboru

In [4]:
#import weights and prices from txt file

def load_input_data(file_path):

    weights = []
    prices = []

    with open(file_path) as file:
        line_split = file.readline().split(" ")
        items_count = int(line_split[0])
        max_weight = int(line_split[1])
        for i in range(items_count):
            line_split = file.readline().split(" ")
            prices.append(int(line_split[0]))
            weights.append(int(line_split[1]))

    return weights, prices, max_weight

### Sedond try - permutation individual

max fitness - berieme itemy v poradí permutácie, kým nenaplníme batoh. Ostatné neberieme

In [5]:
def fitness_batoh_perm(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight > max_weight:
            return total_price,
        total_price += prices[i]
    return total_price,

In [6]:
def fitness_batoh_perm_2(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight <= max_weight:
            total_price += prices[i]
        else:
            total_weight -= weights[i]
    return total_price,

In [7]:
def mut_transposition(individual, indv_size, transposition_count):
    count = random.randint(1, transposition_count)

    for i in range(count):
        pos_1 = random.randint(0, indv_size - 1)
        pos_2 = random.randint(0, indv_size - 1)

        individual[pos_1], individual[pos_2] = individual[pos_2], individual[pos_1]

    return individual,

In [12]:
weights, prices, max_weight = load_input_data(file_path = "./domaci_ukol_data/input_data_1000.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(indv_size), indv_size)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_batoh_perm)

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", mut_transposition, indv_size=indv_size, transposition_count = 20)
#toolbox.register("select", tools.selRoulette)
toolbox.register("select", tools.selTournament, tournsize = 3)

c:\Users\llama\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\llama\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [13]:
POP_SIZE = 3_000
GEN_COUNT = 10_000


pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.4, ngen=GEN_COUNT,
                               stats=stats, halloffame=hof, verbose=True)

print(hof)
print(fitness_batoh_perm(hof[0]))

gen	nevals	avg    	std    	min 	max  
0  	2000  	4655.57	1234.97	1345	10031
1  	1609  	5443.04	1161.78	1704	10031
2  	1613  	6012.49	1201.26	2125	10510
3  	1629  	6356.85	1285.36	1899	10983
4  	1658  	6616.31	1380.04	1515	10614
5  	1632  	6935.38	1434.29	2239	11170
6  	1611  	7240.94	1531.44	2126	11866
7  	1655  	7473.63	1595.53	2057	11866
8  	1627  	7627.57	1667.31	2117	11768
9  	1644  	7838.9 	1706.1 	2332	11768
10 	1619  	7996.76	1797.07	2153	12245
11 	1638  	8154.44	1863.27	1692	12931
12 	1623  	8266.58	1927.16	2189	12931
13 	1658  	8349.04	2001.43	1908	12931
14 	1601  	8572.08	2062.05	2490	13304
15 	1617  	8802.6 	2082.53	2050	14204
16 	1619  	8927.99	2206.49	2083	13979
17 	1650  	9170.47	2167.62	2139	14008
18 	1585  	9482.48	2213.93	2979	14008
19 	1620  	9707.04	2276.98	1319	14385
20 	1616  	10010.1	2325.66	2669	14303
21 	1643  	10309.6	2307.08	2488	14479
22 	1639  	10711.1	2219.7 	2251	14721
23 	1619  	11001.3	2235.62	3329	15504
24 	1637  	11328.3	2114.9 	3065	15504
25 	1613  	1

KeyboardInterrupt: 

In [14]:
print(hof)
print(fitness_batoh_perm(hof[0]))

[[480, 539, 60, 886, 573, 446, 273, 216, 147, 25, 986, 830, 822, 10, 463, 737, 121, 887, 786, 612, 53, 469, 736, 362, 743, 845, 426, 37, 770, 382, 599, 215, 476, 48, 603, 347, 245, 334, 914, 937, 824, 30, 23, 945, 13, 134, 732, 421, 645, 992, 984, 32, 373, 669, 989, 473, 254, 987, 703, 151, 708, 855, 494, 849, 269, 38, 493, 775, 6, 146, 967, 751, 592, 657, 419, 610, 249, 379, 281, 236, 741, 253, 95, 906, 8, 29, 80, 342, 144, 174, 544, 152, 78, 222, 867, 127, 220, 26, 59, 717, 631, 496, 445, 143, 108, 252, 581, 275, 776, 803, 389, 748, 735, 62, 3, 461, 853, 31, 407, 994, 65, 380, 73, 491, 103, 569, 238, 89, 758, 522, 436, 75, 562, 676, 44, 133, 869, 976, 933, 109, 583, 92, 642, 343, 687, 0, 990, 386, 587, 548, 351, 768, 272, 726, 598, 411, 707, 969, 169, 710, 813, 381, 706, 784, 96, 20, 820, 5, 659, 725, 686, 802, 586, 457, 58, 387, 694, 704, 244, 175, 348, 600, 306, 159, 602, 787, 162, 997, 355, 530, 397, 112, 831, 61, 697, 176, 384, 832, 492, 903, 87, 531, 575, 90, 295, 28, 702, 297, 

#### first try - binárny jedinci

max fitness (cena ju zvyšuje a váha prekročená cez max nosnosť batohu prináša postihy)

In [ ]:
def fitness_batoh_bin(individual):
    total_price = np.sum(np.array(individual) @ np.array(prices))
    total_weight = np.sum(np.array(individual) @ np.array(weights))

    if total_weight <= max_weight:
        return total_price,
    else:
        return 1,

Vytvorenie prostredia (knižnica Deap)

In [ ]:
weights, prices = load_input_data(file_path = "./domaci_ukol_data/input_data_100.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, indv_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_batoh_bin)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selRoulette)

Final part

In [ ]:
pop = toolbox.population(n=500)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=200, 
                               stats=stats, halloffame=hof, verbose=True)